In [1]:
import pandas as pd
import numpy as np
import os
import random
import deepchem as dc
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score

import warnings
from rdkit import RDLogger

warnings.filterwarnings("ignore", category=DeprecationWarning)
RDLogger.DisableLog('rdApp.*')

Skipped loading some Pytorch utilities, missing a dependency. No module named 'torch'


This module requires PyTorch to be installed.


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
#Importing ESOL Delaney solubility dataset 

def load_data():
    
    tasks, datasets, transformers = dc.molnet.load_delaney(featurizer="GraphConv", splitter="random", reload=False)
    
    train_dataset, valid_dataset, test_dataset = datasets
    
    train_df = train_dataset.to_dataframe()
    valid_df = valid_dataset.to_dataframe()
    test_df = test_dataset.to_dataframe()
    
    smiles_train, sol_train = train_df["ids"], train_df["y"]
    #smiles_test, sol_test = test_df["ids"], test_df["y"]
    #smiles_valid, sol_valid = valid_df["ids"], valid_df["y"]

    return smiles_train, sol_train #(smiles_valid, sol_valid), (smiles_test, sol_test)

In [3]:
#Featurise data

def featurise_data(smiles, solubility):
    
    featurizer = dc.feat.CircularFingerprint(size=2048, radius=4)
    
    X = featurizer.featurize(smiles)
    y = solubility

    return X, y 

In [4]:
#Train test split 

def splitting(X, y, smiles):

    seed = 400
    random.seed(seed)
    
    X_train, X_test, y_train, y_test, smiles_train, smiles_test = train_test_split(X, y, smiles, test_size=0.2, random_state=seed)

    return X_train, X_test, y_train, y_test, smiles_train, smiles_test, seed

In [5]:
#Train the model

def first_training(X_train, y_train, seed):

    reg = GradientBoostingRegressor(random_state=seed)
    reg.fit(X_train, y_train)
    
    score=(cross_val_score(reg, X_train, y_train, cv=3, n_jobs=-1).mean())
    print(f"Initial Cross-validation score is: {score}")

    return score, reg

In [6]:
#Optional Optimisation

def optional_optimisation(reg, X_train, y_train):

    #Example optimisation parameters. For exhaustive searches use a range of values. 
    
    optimisation_param_grid = {
        "n_estimators": [500],
        "learning_rate": [0.1],
        "max_depth":[7],
        "min_samples_leaf": [4],
        "min_samples_split": [6], 
        "subsample": [0.7]
    }
    
    reg2 = GridSearchCV(reg, optimisation_param_grid, cv=3, n_jobs=-1)
    
    search = RandomizedSearchCV(reg, param_distributions= optimisation_param_grid, n_iter=100, cv=3, n_jobs=-1, verbose=2)
    search.fit(X_train, y_train)

    best_params = search.best_params_
    best_optimisation_score = search.best_score_
    
    print(f"Best Training Parametres: {best_params}")
    print(f"Best Training Score: {best_optimisation_score}")

    return search, reg2, best_params, best_optimisation_score

In [7]:
#Final training  

def final_training(X_train, y_train, best_params, seed=400):

    reg = GradientBoostingRegressor(
        n_estimators=best_params["n_estimators"],
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        min_samples_leaf=best_params["min_samples_leaf"],
        min_samples_split=best_params["min_samples_split"],
        subsample=best_params["subsample"],
        random_state=seed
    )
    
    reg.fit(X_train, y_train)

    print("Final model trained on full training data")

    return reg

In [8]:
#Creating dataframes

def dataframe_creation(reg, y_test, smiles_test, X_train, y_train, smiles_train):

    def build_df(X, y, smiles):
        preds = reg.predict(X)
        residuals = preds - y 
        return pd.DataFrame({"SMILES":smiles, 
                             "Actual Solubility logS [mol/L]": y,
                             "Predicted Solubility logS [mol/L]": preds,
                             "Residuals": residuals
                            })

    train_dataframe = build_df(X_train, y_train, smiles_train)
    test_dataframe = build_df(X_test, y_test, smiles_test)
                             
    return test_dataframe, train_dataframe

In [9]:
#Calculating Scores

def score_calculation(test_dataframe, train_dataframe):

    predicted_RMSE = root_mean_squared_error(test_dataframe["Actual Solubility logS [mol/L]"], test_dataframe["Predicted Solubility logS [mol/L]"])
    predicted_r2_score = r2_score(test_dataframe["Actual Solubility logS [mol/L]"], test_dataframe["Predicted Solubility logS [mol/L]"])
    
    print(f"The predicted RMSE is: {predicted_RMSE}")
    print(f"The predicted R2 score is: {predicted_r2_score}")

    return predicted_RMSE, predicted_r2_score

In [10]:
#Scatter graph 

def scatter_plot(test_dataframe, predicted_RMSE, predicted_r2_score):

    min_val = test_dataframe["Actual Solubility logS [mol/L]"].min()
    max_val = test_dataframe["Actual Solubility logS [mol/L]"].max()
    
    plt.scatter(test_dataframe["Actual Solubility logS [mol/L]"], test_dataframe["Predicted Solubility logS [mol/L]"], s=3) #x, y
    plt.title("Predicted molecular solubility vs measured solubility using gradient boosted trees")
    plt.xlabel("Measured Solubility logS [mol/L]")
    plt.ylabel("Predicted Solubility logS [mol/L]")
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--')
    plt.text(0.9, 0.2, 'R-squared = %.3f\nRMSE = %.3f' % (predicted_r2_score, predicted_RMSE))
    plt.savefig("Scatter Plot")

In [11]:
#Residual Plot 

def residual_plot(test_dataframe):

    sns.residplot(x=test_dataframe["Predicted Solubility logS [mol/L]"], y=test_dataframe["Residuals"])
    plt.title("Residual Plot")
    plt.xlabel("Predicted Solubility logS [mol/L]")
    plt.ylabel("Residuals (Predicted - Actual)")
    plt.axhline(0, color='red', linestyle='--')
    plt.savefig("Residual Plot")

In [12]:
#Feature importance

def feature_importance(reg, X_train):

    importances = reg.feature_importances_ #GB assigns numerical importance to each feature
    X_train_df = pd.DataFrame(X_train, columns=[f"Feature {i}" for i in range(X_train.shape[1])])
    feature_names = X_train_df.columns
    
    feat_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(10,6))
    plt.barh(feat_imp_df['Feature'][:20], feat_imp_df['Importance'][:20])
    plt.gca().invert_yaxis()
    plt.xlabel('Importance')
    plt.title('Top 20 Feature Importances')
    plt.savefig("Feature Importance Plot")
    plt.show()

In [ ]:
#Running the code 

smiles_train, sol_train = load_data()
X, y = featurise_data(smiles_train, sol_train)
X_train, X_test, y_train, y_test, smiles_train, smiles_test, seed = splitting(X, y, smiles_train)
score, reg = first_training(X_train, y_train, seed)
search, reg2, best_params, best_optimisation_score = optional_optimisation(reg, X_train, y_train)
reg = final_training(X_train, y_train, best_params, seed=400)
test_dataframe, train_dataframe = dataframe_creation(reg, y_test, smiles_test, X_train, y_train, smiles_train)
predicted_RMSE, predicted_r2_score = score_calculation(test_dataframe, train_dataframe)
scatter_plot(test_dataframe, predicted_RMSE, predicted_r2_score)
residual_plot(test_dataframe)
feature_importance(reg, X_train)